## Task1 Evaluation of the CNN

In [23]:
import torch
import torchvision as tv
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim 

## Load the CNN from the saved data

In [27]:
path_test='./cats_and_dogs_filtered/validation/'
batch_size_test = 100 
batch_size_train = 100 
num_workers_test = 0 
num_workers_train = 0 
size_compressed = [80,80]
# to load the parameters of the net saved before, construct the net with the same structure at first
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # here we use a neural network contains 2 types of the neural networks, a convolution network for the feature extraction
        # and a fully connected layer to realize the classification 
        # ===========================================================================
        # define the number of filters in the CNNs for features extraction
        num_filters = [1, 64, 64, 128, 128]
        # define the number of the nodes in the fully connected layers for classification
        num_NN = [256, 64]
        self.con1 = nn.Conv2d(in_channels = num_filters[0],
                      out_channels = num_filters[1], 
                      kernel_size = 5, 
                      padding = 1,
                      bias = False)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm2d(num_features=num_filters[1])
        self.con2= nn.Conv2d(in_channels = num_filters[1],
                      out_channels = num_filters[2], 
                      kernel_size = 3, 
                      padding = 1,
                      bias = False)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm2d(num_features=num_filters[2])
        self.con3 = nn.Conv2d(in_channels = num_filters[2],
                      out_channels = num_filters[3], 
                      kernel_size = 3, 
                      padding = 1,
                      bias = False)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm2d(num_features=num_filters[3])
        self.con4 = nn.Conv2d(in_channels = num_filters[3],
                      out_channels = num_filters[4], 
                      kernel_size = 3, 
                      padding = 1,
                      bias = False)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn4 = nn.BatchNorm2d(num_features=num_filters[4])
        self.fc1 = nn.Linear(num_filters[4]*4*4,num_NN[0])
        self.fc2 = nn.Linear(num_NN[0],num_NN[1])
        self.fc3 = nn.Linear(num_NN[1],2)
    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.con1(x))))
        x = self.pool2(F.relu(self.bn2(self.con2(x))))
        x = self.pool3(F.relu(self.bn3(self.con3(x))))
        x = self.pool4(F.relu(self.bn4(self.con4(x))))
        x = x.view(-1, 128*4*4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
kaggle_Net = CNN()
kaggle_Net = torch.load('.\kaggle_Net.pkl')

## Test dataset preprocessing

In [28]:
test_transform = tv.transforms.Compose([
    tv.transforms.Grayscale(num_output_channels=1), # reduce the channel Nr to one
    tv.transforms.Resize(size_compressed),
    tv.transforms.ToTensor()
])
test_data = tv.datasets.ImageFolder(root = path_test, transform = test_transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size_test,
                                           num_workers = num_workers_test, shuffle = True)

## Compute the error rate given test set

In [29]:
errors_test = 0
for batch_idx , data in enumerate(test_loader,0):
    inputs, labels = data
    outputs = kaggle_Net(inputs)
    # error rate
    predicted = torch.max(outputs,dim=1)
    errors_test += sum(predicted[1] != labels)
print('Error rate on test set:', round(100.0* errors_test.numpy() / len(test_loader.dataset),2), '%')

Error rate on test set: 24.1 %
